In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, TimeDistributed, ConvLSTM2D
from tensorflow.keras.utils import to_categorical

In [60]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten

In [62]:
data_path="C:/Users/ESA LAB/Desktop/CNN_FOG/dataset"


In [63]:
'''
people[] -> 存所有在資料夾中的txt檔

'''
people = [] #S01R01.txt~S10R01.txt

for person in os.listdir(data_path):
    if '.txt' in person:
        if 'activity_labels' in person:
            continue
        people.append(person)


In [64]:
'''


'''
def label_prefog(dataset, window_length):
# def label_prefog(dataset, window_length = 1):
    # Drop all the rows for which Action is 0 or the rows which are not part of the experiment
    # 將所有的Action==0(未實驗)的部分刪除，並不回傳任何值(implace的部分)
    dataset.drop(index = list(dataset[dataset['Action'] == 0].index),
                 inplace = True)
    
    window_length = int(64*window_length)
    #print(window_length)
    fog_index = []
    for i in dataset.index:
        if dataset.loc[i, 'Action'] == 2: # 取index和'Action'對應到的資料
            fog_index.append(i) # 將發生fog的index存入fog_index這個array
    fog_index
    
    # print(fog_index)
    
    start_indices = []
    for i in fog_index:
        if (dataset.loc[i-1, 'Action'] != dataset.loc[i, 'Action']):
            start_indices.append(i) # 將發生fog之前的index存入start_indices這個array
    
    # print(start_indices)

    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-window_length, start)] # prefog_start把FOG之前的windows標記起來
        prefog.append(prefog_start)
        
    # print(prefog) # 2D array

    prefog = [item for sublist in prefog for item in sublist] # 把[][]轉換成[]

    # print(prefog) # 1D array

    for i in prefog:
        dataset.loc[i,'Action'] = 3 # 把紀錄在Pre-fog的index action 記錄成3
    dataset['Action'] = dataset['Action'] - 1 # 所有action編號-1 使0=walk 1=fog 2= pre-fog
    return dataset

In [65]:
dataset = pd.DataFrame()
last_name = ''
patients = []
for person in people:
    name = person.split('R')[0]
    if name != last_name:
        dataset = dataset.drop(index=dataset.index)
        patients.append(name)
    last_name = name
    print(name)
    file = data_path + "\\" + person

    if name in person:
        temp = pd.read_csv(file, delimiter = " ", header = None)
        print(person, ' is read', end = "\t")

        #if 2 in temp[max(temp.columns)].unique():
        print("Adding {} to dataset".format(person), end = "\t")
        temp.columns = [
                            'time',
                            'A_F', 'A_V', 'A_L',
                            'L_F', 'L_V', 'L_L',
                            'T_F', 'T_V', 'T_L',
                            'Action'                           
                            ]
        window_length = 2.5
        temp = label_prefog(temp, window_length).reset_index(drop = True)
        temp['name'] = name
        print("{} is labelled".format(person))
        dataset = pd.concat([dataset, temp],axis = 0)
        print('')
    
        dataset.reset_index(drop = True, inplace = True)
        to_path = data_path+"\\raw_labelled"
        to_name = to_path + "\\win_" + name + ".csv"
    # 存入/raw_labelled中
    
    dataset.to_csv(to_name, index = False)

dataset.head()

S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	Adding S03R03.txt to dataset	S03R03.txt is labelled

S04
S04R01.txt  is read	Adding S04R01.txt to dataset	S04R01.txt is labelled

S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled

S05
S05R02.txt  is read	Adding S05R02.txt to dataset	S05R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	Adding S06R02.txt to dataset	S06R02.txt is labelled

S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled


,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
0,600000,-353,950,178,172,953,232,87,1000,-116,0,S10
1,600015,-383,960,198,181,944,212,97,990,-87,0,S10
2,600031,-363,950,178,190,935,232,97,1000,-116,0,S10
3,600046,-363,950,198,172,944,202,97,1000,-97,0,S10
4,600062,-343,960,207,163,944,232,106,1000,-97,0,S10


In [66]:
def create_window(act,window_length,dataframe):

  indices = list(dataframe[dataframe.Action == act].index)
  # indices記錄所有Action==act的index
  groups = [] # 用來暫存一組(同action)資料的, 型態是[][]
  temp = [] # 用來暫存一行資料的
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break # 如果i已經來到最後的話就break
    temp.append(indices[i])
    if indices[i]+1 != indices[i+1]: #如果下個index不是連續的話, 就將前面這些存成第一組
      group_count+=1
      groups.append(temp)
      temp = []

  #print(groups)

  fs = 64
  # window_length = 1
  window_length = int(window_length*fs)

  final_dataframe = pd.DataFrame()
  sumOfAct=0

  for i in groups: # group[][]的每一行i
    required = math.floor(len(i)/(window_length))

    
    sumOfAct= sumOfAct+required

    req_index = i[0:(required*window_length)]

    #print(req_index)
    # concat([要結合的data集合], axis=0是方向為直的)
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  
  

  return final_dataframe

In [67]:
for patient in patients:
  
  path = os.getcwd()+"\\dataset"
  name = path+"\\raw_labelled\\win_"+ patient +".csv"
  dataframe = pd.read_csv(name)

  activities = []
  # print(patient)
  for act in range(3):
    activities.append(create_window(act,2.5,dataframe))
  to_write = pd.concat(activities,axis = 0)
  to_path = path + "\\windows"+"\\windowed_"+patient+".csv"
  to_write.to_csv(to_path,index = False)

In [68]:
total_dataset=pd.DataFrame()
for patient in patients:
  
  path = os.getcwd()+"\\dataset\\windows"
  name = path+"\\windowed_"+ patient +".csv"
  dataframe = pd.read_csv(name)
  total_dataset = total_dataset.append(dataframe)

to_path = path + "\\total.csv"
total_dataset.to_csv(to_path,index = False)

In [69]:
total_dataset.describe

<bound method NDFrame.describe of            time  A_F   A_V  A_L  L_F  L_V  L_L  T_F   T_V  T_L  Action name
0        750000  -30   990  326  -45  972  181  -38  1000   29       0  S01
1        750015  -30  1000  356  -18  981  212  -48  1028   29       0  S01
2        750031  -20   990  336   18  981  222  -38  1038    9       0  S01
3        750046  -20  1000  316   36  990  222  -19  1038    9       0  S01
4        750062    0   990  316   36  990  212  -29  1038   29       0  S01
...         ...  ...   ...  ...  ...  ...  ...  ...   ...  ...     ...  ...
142715  2899906 -505   823  336    0    0    0    0     0    0       0  S10
142716  2899921 -505   823  336    0    0    0    0     0    0       0  S10
142717  2899937 -505   823  336    0    0    0    0     0    0       0  S10
142718  2899953 -505   823  336    0    0    0    0     0    0       0  S10
142719  2899968 -505   823  336    0    0    0    0     0    0       0  S10

[1104960 rows x 12 columns]>

In [70]:
title = ["A_F", "A_V", "A_L", "L_F", "L_T", "L_L", "T_F", "T_V", "T_L"]

In [71]:
total_windows = int((len(total_dataset))/160)
total_windows

6906

In [72]:
path = os.getcwd()+"\\dataset\\final_windowed\\total.csv"

total_dataset = total_dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L", "Action"]].values

切資料

In [73]:
'''
X = np.empty((13811, 160, 9))
y = np.empty((13811, 1))
j = 0

window_count = 0
for items in range(13811):
    for i in range(160):
        if i == 0:
            y[j] = total_dataset[int(window_count*160), 9]
            j = j + 1
        for data in range(9):
            X[items, i, data] = total_dataset[int(window_count*160)+i, data]
    window_count = window_count+0.5
'''

'\nX = np.empty((13811, 160, 9))\ny = np.empty((13811, 1))\nj = 0\n\nwindow_count = 0\nfor items in range(13811):\n    for i in range(160):\n        if i == 0:\n            y[j] = total_dataset[int(window_count*160), 9]\n            j = j + 1\n        for data in range(9):\n            X[items, i, data] = total_dataset[int(window_count*160)+i, data]\n    window_count = window_count+0.5\n'

In [74]:
X = np.empty((6906, 160, 9))
y = np.empty((6906, 1))
j = 0

window_count = 0
for items in range(6906):
    for i in range(160):
        if i == 0:
            y[j] = total_dataset[int(window_count*160), 9]
            j = j + 1
        for data in range(9):
            X[items, i, data] = total_dataset[int(window_count*160)+i, data]
    window_count = window_count+1

In [75]:
y = to_categorical(y)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.25, random_state=42)

In [77]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    n_steps, n_length = 4, 40
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), 
                              input_shape=(None, n_length, n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [78]:
def run_experiment(trainX, trainy, testX, testy, repeats=10):

    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [79]:
run_experiment(X_train, y_train, X_test, y_test, repeats=1)

>#1: 89.114
Accuracy: 89.114% (+/-0.000)


In [ ]:
modelC1 = Sequential()
modelC1.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(160, 9)))
modelC1.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
modelC1.add(Flatten())
modelC1.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
modelC1.add(Dense(units=1))
modelC1.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
print('\n',modelC1.summary())
history = modelC1.fit(X, y, batch_size=32, epochs=25, verbose=0)
    

In [110]:
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    # 對於一維卷積來說，data_format='channels_last'是默認配置，該API的規則如下：
    # 輸入形狀爲：(batch, steps, channels)；輸出形狀爲：(batch, new_steps, filters)，padding和strides的變化會導致new_steps變化
    # 如果設置爲data_format = 'channels_first'，則要求輸入形狀爲： (batch, channels, steps).
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    # 對於一維池化層來說，data_format='channels_last'是默認配置，該API的規則如下：
    # 3D 張量的輸入形狀爲: (batch_size, steps, features)；輸出3D張量的形狀爲：(batch_size, downsampled_steps, features)
    # 如果設置爲data_format = 'channels_first'，則要求輸入形狀爲：(batch_size, features, steps)
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    # data_format參數的作用是在將模型從一種數據格式切換到另一種數據格式時保留權重順序。默認爲channels_last。
    # 如果設置爲channels_last，那麼數據輸入形狀應爲：（batch，…，channels）；如果設置爲channels_first，那麼數據輸入形狀應該爲（batch，channels，…）
    # 輸出爲（batch, 之後參數尺寸的乘積）
    model.add(Flatten())
    
    # Dense執行以下操作：output=activation（dot（input，kernel）+bias），
    # 其中,activation是激活函數，kernel是由層創建的權重矩陣，bias是由層創建的偏移向量（僅當use_bias爲True時適用）。
    # 2D 輸入：(batch_size, input_dim)；對應 2D 輸出：(batch_size, units)
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    # 因爲要預測下一個時間步的值，因此units設置爲1
    model.add(Dense(units=1))
    
    # 配置模型
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    # X爲輸入數據，y爲數據標籤；batch_size：每次梯度更新的樣本數，默認爲32。
    # verbose: 0,1,2. 0=訓練過程無輸出，1=顯示訓練過程進度條，2=每訓練一個epoch打印一次信息
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

In [113]:
sw_width = 160
n_features = 9
epoch_num = 25
verbose_set = 0

In [114]:
model, history = oned_cnn_model(sw_width, n_features, X_train, y_train, X_test, epoch_num, verbose_set)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 159, 64)           1216      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 79, 64)           0         
 g1D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 5056)              0         
                                                                 
 dense_22 (Dense)            (None, 50)                252850    
                                                                 
 dense_23 (Dense)            (None, 1)                 51        
                                                                 
Total params: 254,117
Trainable params: 254,117
Non-trainable params: 0
_______________________________________________